In [ ]:
import pandas as pd

In [ ]:
log_file = "log_01-2022-04-17-21-43-20.csv"
log_df = pd.read_csv(f"LOG_CSV\{log_file}", sep=";", index_col=False)
log_df.reset_index()
log_df.set_index('date', inplace=True)
log_df.index = pd.to_datetime(log_df.index)
log_df.columns

In [ ]:
filter_symbol    = log_df["symbol"] == "ORCL"
filter_buy       = log_df["order_type"] == "Buy"
filter_completed = log_df["order_status"] == "Completed"
filter_accepted  = log_df["order_status"] == "Accepted"
filter_canceled  = log_df["order_status"] == "Canceled"
market_order = log_df["order_name"] == "Market"

#log_df.loc[filter_symbol & (filter_completed | filter_canceled)].head(50)
log_df.loc[filter_symbol & (filter_accepted | filter_completed ) ].head(10)

In [ ]:
def trades_statistics(trades):
    stats = dict()
    stats['symbol'] = trades.iloc[0]['symbol']
    stats['number_of_trades'] = len(trades)
    stats['gain'] = trades['net'].sum()
    
    return stats

In [ ]:
def analyse_trades(symbol, log_df):
    filter_symbol = log_df["symbol"] == symbol
    filter_completed = log_df["order_status"] == "Completed"
    transactions = log_df.loc[filter_symbol & filter_completed]

    t_id = 0
    trades = []
    for i in range(0, len(transactions) - (len(transactions) % 2), 2):
        t_buy  = transactions.iloc[i].copy()
        t_sell = transactions.iloc[i+1].copy()
        days_held = (transactions.index[i+1] - transactions.index[i]).days

        trade = dict()
        trade['trade_id'] = t_id
        trade['symbol'] = t_buy['symbol']
        trade['days_held'] = days_held
        trade['shares'] = t_buy['order_size']
        
        trade['buy_date']   = transactions.index[i]
        trade['buy_order_ref'] = t_buy['order_ref']
        trade['buy_name'] = t_buy['order_name']
        trade['buy_price'] = t_buy['order_price']
        trade['buy_shares'] = t_buy['order_size']
        trade['buy_order_value'] = t_buy['order_value']
        trade['buy_tx_value'] = t_buy['order_size'] * t_buy['order_price'] - t_buy['order_commission']
        
        trade['sell_date'] = transactions.index[i+1]
        trade['sell_order_ref'] = t_sell['order_ref']
        trade['sell_name'] = t_sell['order_name']
        trade['sell_price'] = t_sell['order_price']
        trade['sell_shares'] = t_sell['order_size']
        trade['sell_order_value'] = t_sell['order_value']
        trade['sell_tx_value'] = (t_sell['order_size'] * t_sell['order_price'] * -1) - t_sell['order_commission']

        trade['net'] = trade['sell_tx_value'] - trade['buy_tx_value']

        t_id += 1
        trades.append(trade)
        #buy = dict()
        #buy['date'] = 
        #print(f"[{transactions.index[i]}] {t_buy['symbol']} -> BUY: {t_buy['order_name']} {t_buy['order_type']} | [{transactions.index[i+1]}] {t_sell['symbol']} -> SELL: {t_sell['order_name']} {t_sell['order_type']}")

    trades_df = pd.DataFrame(trades)

    analysis = dict()
    analysis['trades'] = trades_df
    analysis['stats'] = trades_statistics(trades_df)

    return analysis

In [ ]:
all_symbols = list(set(log_df['symbol'].to_list()))
results = []
for symbol in all_symbols:
    result = analyse_trades(symbol, log_df)
    results.append(result)

[stats['stats'] for stats in results]


In [ ]:
results[0]['trades']

In [ ]:
df1 = results[0]['trades']
net_col = df1['net']
net_col.sum()
